---
## Project : 하루시작 지하철 혼잡도 분석 02 
## Description : 
    - 서울 교통공사 역별 일별 시간대별 승하차인원 정보
    [source link] (https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do#)
    
## Author : Forrest Dpark (분석 담당)
## Date : 2024.05.31 ~
## Detail : 
## Update: 
    - 2024.06.02 pdg : 일별 데이터 정제 
        - 기존 테스트 데이터 의 일별 데이터가 없는 부분 참고하여 일별 데이터 추출 및 데이터 칼럼 정제를 시작.


In [1]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
def plotSetting(pltStyle):
    import matplotlib.pyplot as plt # visiulization
    import platform
    from matplotlib import font_manager, rc # rc : 폰트 변경 모듈font_manager : 폰트 관리 모듈
    plt.style.use(pltStyle)
    plt.rcParams['axes.unicode_minus'] = False# unicode 설정
    if platform.system() == 'Darwin': rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows': # os가 windows
        path = 'c:/Windows/Fonts/malgun.ttf' 
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")
    print("___## OS platform 한글 세팅완료 ## ___")
# graph style seaborn
plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [2]:
# Data loading 
subway = pd.read_csv("../Data/교통공사데이터/서울교통공사_역별 일별 시간대별 승하차인원 정보_20221231.csv", encoding='euc-kr')
subway.head()

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,1,2022-01-01,1,150,서울역,승차,120,137,211,439,...,1566,1686,1591,1358,1062,899,1327,814,234,NaN
1,2,2022-01-01,1,150,서울역,하차,113,560,617,910,...,1329,1251,1126,884,764,654,728,416,131,NaN
2,3,2022-01-01,1,151,시청,승차,38,66,101,139,...,474,550,672,528,420,434,491,232,38,NaN
3,4,2022-01-01,1,151,시청,하차,31,195,224,380,...,408,377,354,213,131,98,137,61,24,NaN
4,5,2022-01-01,1,152,종각,승차,44,71,86,158,...,889,964,1024,803,855,1099,1209,255,62,NaN


In [3]:
def dataInfoProcessing(df, replace_Nan=False):
    ''' 
    # Fucntion Description :  Data frame 의 정제해야할 부분을 체크해주는 함수 
    # Date : 2024.06.02 
    # Author : Forrest D Park 
    # update : 
        * 2024.06.02 by pdg: 일별 데이터 정제 
            - 데이터에 null 이 있음을 발견, data 정제 함수 update 
            - 함수에서 replace_Nan 아규 멘트 받아서 true 일경우 nan 을 0 으로 대체 하게 만듬. 
    '''
    print("\n1. Data colum numbers : ",len(df.columns))
    #print(subway.columns)
    #print(subway.info())
    null_message =f"총 {df.isnull().sum().sum()}개의 null 이 있습니다!" if df.isnull().sum().sum() else "Null 없는 clean data!"
    print("\n2. null ceck 결과",null_message)
    ### Null 이 있는 칼럼 추출
    haveNullColumn =[]
    for idx, col in enumerate(df.columns):
        if df[f"{col}"].isnull().sum():
            print(f"   => {idx}번째.[{col}]컬럼 : ",f"null {df[f'{col}'].isnull().sum()} 개,\t not null {df[f'{col}'].notnull().sum()} 개")
            ## Null data fill
    if replace_Nan : ## nan 을 0 으로 대체 
        df[col].fillna(value=0, inplace=True)  
    
    print("\n3. Column  Information (중복체크)")
    print( "\tidx.columName |\t\t\t\t |Colum Info(dtype)|** ")
    print( "\t","--"*len("columIdx |\t\t\t\t **|Col(dtype)|** "))
    for idx, col in enumerate(df.dtypes.keys()):
        if idx< 7:
            if len(f"\t{idx}.[{col}({df.dtypes[col]})]:")<20:
                print(f"\t{idx}.[{col}({df.dtypes[col]})]:",\
                    f"{len(df[col].unique())}/{len(df[col])} [uniq/raw]", sep=" \t\t\t")
            else:
                    print(f"\t{idx}.[{col}({df.dtypes[col]})]:",\
                    f"{len(df[col].unique())}/{len(df[col])} [uniq/raw]", sep=" \t\t")

    else: 
        print(f"\t ...etc (추가로 {len(df.dtypes.keys())-5}개의 칼럼이 있습니다 )")
   
              
dataInfoProcessing(subway)

# subway.info(show_counts=False)
# subway.value_counts('수송일자')






1. Data colum numbers :  26

2. null ceck 결과 총 82346개의 null 이 있습니다!
   => 25번째.[24시이후]컬럼 :  null 82346 개,	 not null 116734 개

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			199080/199080 [uniq/raw]
	1.[수송일자(object)]: 			365/199080 [uniq/raw]
	2.[호선(int64)]: 			8/199080 [uniq/raw]
	3.[고유역번호(외부역코드)(object)]: 		558/199080 [uniq/raw]
	4.[역명(object)]: 			244/199080 [uniq/raw]
	5.[승하차구분(object)]: 			2/199080 [uniq/raw]
	6.[06시이전(int64)]: 			1596/199080 [uniq/raw]
	 ...etc (추가로 21개의 칼럼이 있습니다 )


> 25번째 칼럼인 24시 이후 데이터의 null 은 운행을 하지 않아서 null 인가?

In [7]:
interesting_colnumbers = [i for i in range(1,7)] + [-2,-1]
print(interesting_colnumbers)
subway[subway['24시이후'].isnull()].iloc[:,interesting_colnumbers].head()

[1, 2, 3, 4, 5, 6, -2, -1]


,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,23-24시간대,24시이후
0,2022-01-01,1,150,서울역,승차,120,234,NaN
1,2022-01-01,1,150,서울역,하차,113,131,NaN
2,2022-01-01,1,151,시청,승차,38,38,NaN
3,2022-01-01,1,151,시청,하차,31,24,NaN
4,2022-01-01,1,152,종각,승차,44,62,NaN


In [8]:
havenulldate = subway[subway['24시이후'].isnull()].iloc[:,a]
havenulldate['수송일자'].unique()[[1,2,-2,-1]]
# 2022 년 1 월 1일부터  5월 31 일까지 는 12시 이후 영업을 안한 모양.. -> 코로나 단축운행!! 


NameError: name 'a' is not defined

> Nan 을 0 으로 대체 

In [9]:

dataInfoProcessing(subway,replace_Nan=True)


1. Data colum numbers :  26

2. null ceck 결과 총 82346개의 null 이 있습니다!
   => 25번째.[24시이후]컬럼 :  null 82346 개,	 not null 116734 개

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			199080/199080 [uniq/raw]
	1.[수송일자(object)]: 			365/199080 [uniq/raw]
	2.[호선(int64)]: 			8/199080 [uniq/raw]
	3.[고유역번호(외부역코드)(object)]: 		558/199080 [uniq/raw]
	4.[역명(object)]: 			244/199080 [uniq/raw]
	5.[승하차구분(object)]: 			2/199080 [uniq/raw]
	6.[06시이전(int64)]: 			1596/199080 [uniq/raw]
	 ...etc (추가로 21개의 칼럼이 있습니다 )


>  해당 데이터는 8개의 호선이 있고 역번호는 558개, 역명은 244 개가 있다.. 왜?  역번호와 역 명은 1:1 이어야 하는게 아닌가?

In [ ]:
subway['역명']

0         서울역
1         서울역
2          시청
3          시청
4          종각
         ... 
199075     수진
199076     모란
199077     모란
199078    남위례
199079    남위례
Name: 역명, Length: 199080, dtype: object

In [11]:
## 역명 조회
# 지하철 역 테이블 과 역번호 를 따로 만들어서 관리하도록 하자. csv 저장 필요 
역이름 = []
호선 = []
역코드 =[]


for i in subway['역명'].unique():
    for 순서, 역명 in enumerate(subway['역명']):
        if 역명==i :
            # print(f"{subway['호선'][순서]}호선 {i}, {subway['고유역번호(외부역코드)'][순서]}")
            # seoulSubwayInfo[f'{i}']= [f"{subway['호선'][순서]}호선", f"{subway['고유역번호(외부역코드)'][순서]}"]
            역이름.append(i)
            호선.append(f"{subway['호선'][순서]}호선")
            역코드.append(int(f"{subway['고유역번호(외부역코드)'][순서]}"))
            break


In [12]:
## subway column  csv 로 저장 
df= pd.DataFrame(
    {   
    "역코드" : 역코드,
    "호선" : 호선,
    "역이름" : 역이름,
    }, 
)
df.to_csv("../Data/SubwayInfo.csv", index=None)
df.pivot_table(
    # columns='호선'
    index='호선',
    aggfunc=np.size
)

,역이름,역코드
호선,,
1호선,10,10
2호선,48,48
3호선,30,30
4호선,22,22
5호선,49,49
6호선,30,30
7호선,40,40
8호선,15,15


--- 
> 역코드 unique 는 왜 500 개가 넘을까?
> 승차에는 없는 띄어쓰기가 하차에는 있음! 그리고 132480번째 데이터   에는 띄어쓰기가 들어가있음. 

---
###  마지막 수송 일자를 기준으로 역 이름과 역 코드를 판별한다. 



In [ ]:

print("마지막 수송일자 :",list(subway['수송일자'])[-1])

마지막 수송일자 : 2022-12-31


In [13]:
### 역코드 obj -> int 로 변환  ** 아무것도 없는 데이터는 000 으로 변환 
new_stationCode = []
for i, code in enumerate(subway['고유역번호(외부역코드)']):
    # print(str(i).replace(" ",""))
    try : 
        new_stationCode.append(int(str(code)))
    except ValueError : 
        print(f"{i}번째 데이터 {code}" ,"<-value errer: ")
        new_stationCode.append(000)
        continue
print("대체됨" if len(new_stationCode)== len(subway)else "대체 안됨")

subway['고유역번호(외부역코드)'] = new_stationCode
len(subway['고유역번호(외부역코드)'].unique())
# subway.info()
## 승차데이터만, 그리고 마지막 수송일자를 기준으로 칼럼정제
test = subway[['역명','고유역번호(외부역코드)','승하차구분']][subway['승하차구분']=='승차'][subway['수송일자']==list(subway['수송일자'])[-1]]


# list(test.index)
test=test.reset_index()
test.drop(columns='index',inplace=True)
test
# test.to_csv(f"../Data/CurrentSubwayStation_{list(subway['수송일자'])[-1]}.csv",)


132480번째 데이터   <-value errer: 
대체됨


,역명,고유역번호(외부역코드),승하차구분
0,서울역,150,승차
1,시청,151,승차
2,종각,152,승차
3,종로3가,153,승차
4,종로5가,154,승차
...,...,...,...
267,단대오거리,2824,승차
268,신흥,2825,승차
269,수진,2826,승차
270,모란,2827,승차


In [ ]:
## 역명과 고유역번호가 일치하는가?
print("역명개수:",len(test['역명'].unique()),"고유역번호개수",len(test['고유역번호(외부역코드)'].unique()))
test['역명'].value_counts()
# type(a)
test[test['역명']=='종로3가']

역명개수: 239 고유역번호개수 272


,역명,고유역번호(외부역코드),승하차구분
3,종로3가,153,승차
70,종로3가,319,승차
143,종로3가,2535,승차


> 종로 3가 는 1호선, 3호선, 5호선 세 종류의 호선마다 역이 있다.     
> 중복문제를 피하기 위해서 역_이름코드 라는 칼럼을 하나 만들어서 primary key 로 만들자

In [ ]:



# 역이름_코드 칼럼 생성하여 csv 저장 
station_code= test['고유역번호(외부역코드)'].to_numpy()
station_name = test['역명'].to_numpy()
test['역_이름코드'] = station_name_code = [f"{name}_{code}" for name,code in zip(station_name,station_code)]
test.to_csv(f"../Data/StationInfo_{list(subway['수송일자'])[-1]}.csv",)
    
test

,역명,고유역번호(외부역코드),승하차구분,역_이름코드
0,서울역,150,승차,서울역_150
1,시청,151,승차,시청_151
2,종각,152,승차,종각_152
3,종로3가,153,승차,종로3가_153
4,종로5가,154,승차,종로5가_154
...,...,...,...,...
267,단대오거리,2824,승차,단대오거리_2824
268,신흥,2825,승차,신흥_2825
269,수진,2826,승차,수진_2826
270,모란,2827,승차,모란_2827


In [15]:
test.head()

,역명,고유역번호(외부역코드),승하차구분
0,서울역,150,승차
1,시청,151,승차
2,종각,152,승차
3,종로3가,153,승차
4,종로5가,154,승차


In [27]:
subway_6호선_승차 = subway[(subway['호선']== 6) & (subway['승하차구분'] == '승차')]
subway_6호선_승차

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
350,351,2022-01-01,6,2611,응암,승차,152,207,265,388,...,580,594,521,359,271,226,225,165,42,0.0
352,353,2022-01-01,6,2612,역촌,승차,30,55,61,79,...,128,132,89,63,39,51,44,25,8,0.0
354,355,2022-01-01,6,2613,불광,승차,17,42,46,66,...,229,277,212,173,131,124,135,73,9,0.0
356,357,2022-01-01,6,2614,독바위,승차,23,47,50,72,...,74,98,100,50,41,22,22,14,6,0.0
358,359,2022-01-01,6,2616,구산,승차,64,99,119,169,...,251,231,226,117,69,89,93,43,11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198950,198951,2022-12-31,6,2644,돌곶이,승차,143,185,275,452,...,340,393,333,294,194,187,152,129,59,105.0
198952,198953,2022-12-31,6,2645,석계,승차,170,252,365,543,...,575,491,496,371,297,266,218,260,97,20.0
198954,198955,2022-12-31,6,2646,태릉입구,승차,80,100,132,186,...,289,297,258,260,135,150,127,101,41,15.0
198956,198957,2022-12-31,6,2647,화랑대(서울여대입구),승차,82,169,276,536,...,418,456,375,309,163,152,120,106,33,10.0


In [41]:
subway_6호선_승차_slice = subway_6호선_승차.iloc[:, [2, 4] + list(range(6, subway.shape[1]))]


,호선,역명,06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,10-11시간대,11-12시간대,12-13시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
350,6,응암,152,207,265,388,507,478,578,594,...,580,594,521,359,271,226,225,165,42,0.0
352,6,역촌,30,55,61,79,114,88,136,132,...,128,132,89,63,39,51,44,25,8,0.0
354,6,불광,17,42,46,66,87,86,126,152,...,229,277,212,173,131,124,135,73,9,0.0
356,6,독바위,23,47,50,72,101,82,115,128,...,74,98,100,50,41,22,22,14,6,0.0
358,6,구산,64,99,119,169,186,209,251,261,...,251,231,226,117,69,89,93,43,11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198950,6,돌곶이,143,185,275,452,454,425,447,498,...,340,393,333,294,194,187,152,129,59,105.0
198952,6,석계,170,252,365,543,585,592,649,644,...,575,491,496,371,297,266,218,260,97,20.0
198954,6,태릉입구,80,100,132,186,212,260,288,318,...,289,297,258,260,135,150,127,101,41,15.0
198956,6,화랑대(서울여대입구),82,169,276,536,462,497,508,594,...,418,456,375,309,163,152,120,106,33,10.0


In [ ]:
subway_6호선_승차_slice.pivot_table()

In [30]:
subway_6호선_승차_slice

,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,10-11시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
350,6,2611,응암,승차,152,207,265,388,507,478,...,580,594,521,359,271,226,225,165,42,0.0
352,6,2612,역촌,승차,30,55,61,79,114,88,...,128,132,89,63,39,51,44,25,8,0.0
354,6,2613,불광,승차,17,42,46,66,87,86,...,229,277,212,173,131,124,135,73,9,0.0
356,6,2614,독바위,승차,23,47,50,72,101,82,...,74,98,100,50,41,22,22,14,6,0.0
358,6,2616,구산,승차,64,99,119,169,186,209,...,251,231,226,117,69,89,93,43,11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198950,6,2644,돌곶이,승차,143,185,275,452,454,425,...,340,393,333,294,194,187,152,129,59,105.0
198952,6,2645,석계,승차,170,252,365,543,585,592,...,575,491,496,371,297,266,218,260,97,20.0
198954,6,2646,태릉입구,승차,80,100,132,186,212,260,...,289,297,258,260,135,150,127,101,41,15.0
198956,6,2647,화랑대(서울여대입구),승차,82,169,276,536,462,497,...,418,456,375,309,163,152,120,106,33,10.0


In [42]:
subway_6호선_승차.head()

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
350,351,2022-01-01,6,2611,응암,승차,152,207,265,388,...,580,594,521,359,271,226,225,165,42,0.0
352,353,2022-01-01,6,2612,역촌,승차,30,55,61,79,...,128,132,89,63,39,51,44,25,8,0.0
354,355,2022-01-01,6,2613,불광,승차,17,42,46,66,...,229,277,212,173,131,124,135,73,9,0.0
356,357,2022-01-01,6,2614,독바위,승차,23,47,50,72,...,74,98,100,50,41,22,22,14,6,0.0
358,359,2022-01-01,6,2616,구산,승차,64,99,119,169,...,251,231,226,117,69,89,93,43,11,0.0


In [66]:
# 수송일자 날짜형으로 변환
import pandas as pd

## 요일 변경 갯수 확인 13594개
print(len(pd.to_datetime(subway_6호선_승차['수송일자'], format='%Y-%m-%d').dt.day_name()))
print(len(subway_6호선_승차)) 


## 요일 컬럼 생성
subway_6호선_승차['요일'] = pd.to_datetime(subway_6호선_승차['수송일자'], format='%Y-%m-%d').dt.day_name().values
subway_6호선_승차.info() ## 요일 null값 체크 -> 이상없음

# 요일을 영어에서 한국어로 변환
day_name_mapping = {
    'Monday': '월요일',
    'Tuesday': '화요일',
    'Wednesday': '수요일',
    'Thursday': '목요일',
    'Friday': '금요일',
    'Saturday': '토요일',
    'Sunday': '일요일'
}


subway_6호선_승차['요일'] = subway_6호선_승차['요일'].map(day_name_mapping)


13594
13594
<class 'pandas.core.frame.DataFrame'>
Index: 13594 entries, 350 to 198958
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            13594 non-null  int64  
 1   수송일자          13594 non-null  object 
 2   호선            13594 non-null  int64  
 3   고유역번호(외부역코드)  13594 non-null  int64  
 4   역명            13594 non-null  object 
 5   승하차구분         13594 non-null  object 
 6   06시이전         13594 non-null  int64  
 7   06-07시간대      13594 non-null  int64  
 8   07-08시간대      13594 non-null  int64  
 9   08-09시간대      13594 non-null  int64  
 10  09-10시간대      13594 non-null  int64  
 11  10-11시간대      13594 non-null  int64  
 12  11-12시간대      13594 non-null  int64  
 13  12-13시간대      13594 non-null  int64  
 14  13-14시간대      13594 non-null  int64  
 15  14-15시간대      13594 non-null  int64  
 16  15-16시간대      13594 non-null  int64  
 17  16-17시간대      13594 non-null  int64  
 18  17-18시간대      13

In [77]:
data2 = pd.read_csv("../Data/station_time_distance.csv", encoding='euc-kr')
data2[data2['호선']==4]

,호선,역명,시간(분),거리(km),누계(km)
96,4,당고개,0:00,0.0,0.0
97,4,상계,1:30,1.2,1.2
98,4,노원,1:30,1.0,2.2
99,4,창동,2:00,1.4,3.6
100,4,쌍문,1:30,1.3,4.9
101,4,수유,2:00,1.5,6.4
102,4,미아,1:30,1.4,7.8
103,4,미아사거리,2:00,1.5,9.3
104,4,길음,1:30,1.3,10.6
105,4,성신여대입구,2:00,1.4,12.0
